In [1]:
# %pip install pandas requests bs4 numpy


In [2]:
import os
from bs4 import BeautifulSoup
import csv
import requests


In [3]:
url = "https://www.byupathway.edu/policies/handbook/"
index_path = "../data/data_16_09_24/index/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


In [4]:
print(type(soup))


<class 'bs4.BeautifulSoup'>


In [5]:
sections = soup.select("bsp-book>ul>li>bsp-chapter>ul>li>bsp-chapter", class_="Chapter")
print(len(sections))


12


In [6]:
import pandas as pd

data = []
for section in sections:
    div = section.find("div", class_="Chapter-title")

    # does the div exist?
    if div:
        # if yes, is the first-child a span or an a tag?
        span = div.find("span", class_="Link")
        anchor = div.find("a", class_="Link")

        # name the section depending on the firstchild
        if span:
            section_name = span.text.strip()
        elif anchor:
            section_name = anchor.text.strip()
        else:
            section_name = None
        # print(section_name)
        links = section.find_all("a", class_="Link")
        for link in links:
            title_span = link.find("span", class_="Link-span")
            title = title_span.text.strip() if title_span else ""
            url = link["href"]
            data.append([section_name, title, url])


# DataFrame to CSV
df = pd.DataFrame(data, columns=["Section", "Title", "URL"])
# df.to_csv('links.csv', index=False)


In [7]:
# df = pd.read_csv('links.csv')
df.insert(1, "Subsection", "")
# Filter out rows with '#' in the URL
df_filtered = df[~df["URL"].str.contains("#")]

df_filtered.to_csv("stdhndbk.csv", index=False)


In [8]:
df = pd.read_csv("stdhndbk.csv")

# Filter out rows with '#' in the URL
df_filtered = df[~df["URL"].str.contains("#")]
# Save the filtered DataFrame back to CSV
df_filtered.to_csv(index_path + "stdhndbk.csv", index=False)
